In [47]:
import pandas as pd
import numpy as np
from preprocess import scale
from sklearn.linear_model import LogisticRegression

In [39]:
url = 'https://raw.githubusercontent.com/probml/pmtkdata/master/spamData/spam.data'
data = pd.read_table(url, header = None, sep = ' ')

In [40]:
y = data[57].values
X = data[range(57)]
del data

In [ ]:
clf = LogisticRegression()

In [54]:
def test(X):
    clf.fit(X, y)
    print '%0.3f' % (1.0 - np.mean(clf.predict(X) == y))
    
test(scale(X))
test(np.log(X + 0.1))
test( 1 * (X > 0))

0.070
0.054
0.066
